# This keyword
A tutorial on the use of the *this* keyword in JavaScript

## Top Level 
At the very top level the _this_ keyword refers to the **global** object in a Node REPL or the **window** object in the browser. In a Node module it refers to **module.exports**.

In [3]:
// Top level Node REPL
this === global;

true

In [ ]:
// Top level browser
this === window; //only works in a browser

In [4]:
// Top level Node module
this === module.exports;

false

## Plain Functions
In plain functions, _this_ behaves differently depending on `'strict mode'`.
When strict mode is off, _this_ refers to the global object.

In [5]:
function notInStrictMode(){
    return this === global;
}
notInStrictMode();

true

When strict mode is on, _this_ is undefined. This is the same
whether the strict mode is defined in the function or in the 
function's context.

In [6]:
function inStrictMode(){
    'use strict';
    return this === undefined;
}
inStrictMode();

true

### Explanation
This behaviour is enforced to help you avoid putting values into the global space by mistake. 
Consider the constructor function below. 

In [7]:
function Vehicle(make, model){
    this.make = make;
    this.model = model;
}


If this function is invoked without using the new keyword
what does _this_ refer to? Since _this_ in the Vehicle function
refers to the **global** object, the make and model become
global keys.

In [11]:
const veh = Vehicle("Toyota", "Corolla");
console.log('What does variable veh conatin: ', veh);
console.log('global.make: ', global.make);
console.log('global.model: ', global.model);

What does variable veh conatin:  undefined
global.make:  Toyota
global.model:  Corolla


The intention of course is to invoke this with the **new** keyword
in order to construct an object.

In [12]:
const vObj = new Vehicle("Toyota", "Corolla");
console.log(vObj);

Vehicle { make: 'Toyota', model: 'Corolla' }


This makes a good argument for always using **strict** mode.
Accidentally invoking the function will throw an appropriate error.

In [14]:
function StrictVehicle(make, model){
    'use strict';
    this.make = make;
    this.model = model;
}

const errV = StrictVehicle("Toyota", "Corolla");

TypeError: Cannot set property 'make' of undefined

## Methods
A method is a function that is a property of an object. **this** usually refers to
the object on which the method is invoked.